In [1]:
#Versión para tweepy V2

#agregar que tipo de tweet es y a quien responden si es necesario
import datetime
from datetime import date
import tweepy
import csv
import pandas as pd
import json
from subprocess import check_output
import time
from pathlib import Path
import os

# CONSTANTES
PETITIONS_PER_TIME_LAPSE = 300
TIME_LAPSE = 15*60
TIMER = TIME_LAPSE/PETITIONS_PER_TIME_LAPSE
TIMER_LOG = 2*60
listaResultados = []


iniTimerLog = 0
finTimerLog = 0
tiempo = TIMER_LOG

today = datetime.date.today()
# dd/mm/YY
d1 = today.strftime("%d-%m-%Y")
#Crear ficheros de log


fileObj = Path("./log" + d1 + ".txt")
if (fileObj.is_file()):
    os.remove("./log" + d1 + ".txt")
    

fileObj = Path("./errorlog" + d1 + ".txt")
if (fileObj.is_file()):
    os.remove("./errorlog" + d1 + ".txt")

log = open("./log" + d1 + ".txt", "a")
log.write("Primer log\n")
#Crear ficheros de log
errorlog = open("./errorlog" + d1 + ".txt", "a")


#####Uso de Hashtag
f =  open("./hashtags.txt", "r")
hashtagsList = []
hashtag = None
while True:
    # read line
    try:
        hashtag = f.readline()
        if (hashtag == "") :
            break
    except:
        # datetime object containing current date and time
        now = datetime.datetime.now()
        # dd/mm/YY H:M:S
        dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
        errorlog.write("Fallo al leer el fichero hashtags.txt " + dt_string + "\n")

    hashtag = hashtag.rstrip('\n')
    hashtag = hashtag.replace('#', '')
    initDate = f.readline().rstrip('\n')
    lastDate = f.readline().rstrip('\n')
    hashtag = (hashtag, initDate, lastDate)
    hashtagsList.append(hashtag)
    # check if line is not empty
    
f.close()
for h in hashtagsList:
    print(h)



('aldjfolajsdfa', '01-02-2021', '16-04-2021')
('asfasdfasdfasdfa', '01-02-2021', '16-04-2021')
('PSOE', '01-02-2021', '09-04-2021')
('SiYoFueraDeMadrid', '16-04-2021', '16-04-2021')
('SosoSerioYFormal', '01-02-2021', '09-04-2021')
('DemocraciaOFascismo4M', '01-02-2021', '25-04-2021')


In [2]:

#agregar que tipo de tweet es y a quien responden si es necesario
import tweepy
import csv
import pandas as pd
import json
from subprocess import check_output
import time

def DayMonthYearToPetition(date):
    splited = ''
    if ('/' in date):
        splited = date.split('/')
    elif ('-' in date):
        splited = date.split('-')

    return (splited[2] + '-' + splited[1] + '-' + splited[0])

def PetitionToDayMonthYear(petition):
    splited = ''
    time = ''
    splited = petition.split('T')
    time = splited[1]
    splited = splited[0].split('-')
    return (splited[2] + '-' + splited[1] + '-' + splited[0] + 'T' + time)


# Problemas a resolver: 
# - Hay que resolver el problema de la situaci'on geogr'afica y el uso de entities 
# - Hacer el Log y ejecutar durante horas para que muestre dicho Log

# Hashtag: nombre del hashtag que se va a buscar
# start: fecha de inicio
# end: fecha de respuesta
# number: n'umero de tweets que queremos obtener
# bearer: bearer id
# next_token: token de la page (inicialmente string vac'ia) que quieres buscar
# return tupla de n'umero de respuestas y next_token
def PetitionsLessEqual100(hashtag, start, end, number, bearer, next_token):

    startDate = DayMonthYearToPetition(start)
    endDate = DayMonthYearToPetition(end)
    total = (0, '')
    dt_string = ''

    # Insertados los siguientes campos:
    #   -start_time: límite inferior en búsqueda de fechas
    #   -end_time: límete superior en búsqueda de fechas
    #   -tweet.fields=author_id: id del autor del tweet
    #   -tweet.fields=referenced_tweets: puede ser quote, retweet o replied_to junto al id al que responde
    #   -tweet.fields=created_at: cu'ando se cre'o el fichero
    #   -tweet.fields=context_annotations:anotaciones de contexto (no salen muchas)
    #   -tweet.fields=lang: Abreviatura del idioma en el que se escribe
    #   -tweet.fields=entities: referencias, hashtags usados por el usuario
    #   -expansions=geo.place_id: campo necesario para el uso de place.fields=country
    #   -place.fields=country: te indica el pa'is en el que se hizo el tweet si est'a activado la localizaci'on del usuario.
    
    entradaizq = url + "query=%23" + hashtag + "&start_time=" +  startDate + "T00%3A00%3A00Z" + "&end_time=" + endDate + 'T11%3A59%3A59Z&max_results=' + str(number) + '&tweet.fields=author_id,referenced_tweets,created_at,context_annotations,lang,entities&expansions=geo.place_id&place.fields=country'
    entradader = '"' + ' -H "Authorization: Bearer '+ bearer + '"'

    if (next_token != ''):
        entrada = entradaizq + '&next_token=' + next_token + entradader
    else:
        entrada = entradaizq + entradader

    #primera petici'on del hashtag
    json_obj = json.loads(check_output(entrada, shell=True).decode())

    #inicio de timer
    #timerasofa
    # Comprobar que hay respuestas para dicho hashtag
    if ('meta' in json_obj) and (int(json_obj['meta']['result_count']) > 0):

        for tweet in json_obj['data']: 
            #imprimir id de usuario
            print("     author_id: " + tweet['author_id'])
            #imprimir id de tweet
            print("     id: " + tweet['id'])
            #imprimir fecha de creción
            print("       created_at: " + PetitionToDayMonthYear(tweet['created_at']))
            
            if ('referenced_tweets' in tweet):
                #imprimir tweets referidos
                print("       referenced_tweets: " + str(tweet['referenced_tweets']))
            if ('context_annotation' in tweet):
                #imprimir context_annotation
                print("       context_annotation: " + str(tweet['context_annotations']))
            if ('geo' in tweet) and (('includes' in json_obj) and ('places' in json_obj['includes'])):
                place_id = tweet['geo']['place_id']
                for place in json_obj['includes']['places']:
                    if (place_id == place['id']):
                        #imprimir context_annotation
                        print("       Country: " + str(place['country']))
            print("       lang: " + str(tweet['lang']) + '\n')
            print("       entities: " + str(tweet['entities']) + '\n')




        if ('next_token' in json_obj['meta']):
            total = (int(json_obj['meta']['result_count']), json_obj['meta']['next_token'])
        else:
            total = (int(json_obj['meta']['result_count']), '')
        print("N'umero de peticiones enviadas: " + str(number))
        print("N'umero de peticiones recibidas: " + str(json_obj['meta']['result_count']))
        #df = pd.DataFrame(msgs)
        return total

    #si no tiene meta entonces es un error y hay que manejarlo
    elif ('meta' in  json_obj) and (int(json_obj['meta']['result_count']) == 0):
        # datetime object containing current date and time
        now = datetime.datetime.now()
        # dd/mm/YY H:M:S
        dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
        errorlog.write("Error, no hay tweets para el Hashtag: " + str(hashtag) + " " + dt_string + '\n')
        print("No hay tweets para el Hashtag: " + str(hashtag))
    else:
        strerror = ''
        print("Error en la petición: ")
        strerror = strerror + "Error en la petición: "
        # si el error es por cuestión de peticiones esperar para hacer la siguiente peticion
        if ('title' in json_obj):
            print("  Título del error: " + json_obj['title'])
            strerror = strerror + "  Título del error: " + str(json_obj['title'])
            if ('status' in json_obj):
                strerror = strerror + " Código de error: " + str(json_obj['status'])
                print("  Código de error: " + json_obj['status'])
            if ('detail' in json_obj):
                strerror = strerror + "  Descripción del error: " + str(json_obj['detail'])
                print("  Descripción del error: " + json_obj['detail'])

            if ('Invalid Request' == json_obj['title']):
                strerror = strerror + " Error: " + str(entrada.split(' -H')[0])
                print(entrada.split(' -H')[0])
        errorlog.write(strerror + " " + dt_string + '\n')
    return total


# Hashtag: nombre del hashtag que se va a buscar
# start: fecha de inicio
# end: fecha de respuesta
# number: n'umero de tweets que queremos obtener
# bearer: bearer id
# return devuelve el totl de tweets que se han obtenido
def TweetList(hashtag, start, end, number, bearer):
    global tiempo
    global listaResultados
    total = 0
    aux = 0
    resultado = (0, '')
    startTime = 0
    endTime = 0

    print("Inicio Hashtag: " + hashtag + "\\\\\\\\\\\\ \n")
    if (number < 100):
        startTime = time.time()
        total =  PetitionsLessEqual100(hashtag, start, end, number, bearer, None)[0]
        listaResultados.append((hashtag, total))
        endTime = time.time()

        aux = endTime - starTime
        

        if (aux < TIMER):
            time.sleep(math.ceil(TIMER - aux))
        
        tiempo -= aux

        if (tiempo <= 0):
            #aqui se imprime en el log los hashtags que se han escrito y c'uantos valores se han obtenido de estos
            tiempo = TIMER_LOG
            # datetime object containing current date and time
            now = datetime.datetime.now()
            # dd/mm/YY H:M:S
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            log.write(str(listaResultados) + dt_string + '\n')
            listaResultados = []
    else:
        startTime = 0
        endTime = 0
        while ((number - total) > 0):
            #Caso base
            if ((number - total) >= 100):
                startTime = time.time()
                resultado = PetitionsLessEqual100(hashtag, start, end, 100, bearer, resultado[1])
                endTime = time.time()
            #Fin de caso
            elif ((number - total) > 0) and ((number - total) < 100):
                startTime = time.time()
                resultado = PetitionsLessEqual100(hashtag, start, end, (number - total), bearer, resultado[1])
                endTime = time.time()

            aux = endTime - startTime
            if (aux < TIMER):
                time.sleep(math.ceil(TIMER - aux))
                
            tiempo -= aux

            total += resultado[0]
            listaResultados.append((hashtag, total))
            if (tiempo <= 0):
                #aqui se imprime en el log los hashtags que se han escrito y c'uantos valores se han obtenido de estos
                tiempo = TIMER_LOG
                # datetime object containing current date and time
                now = datetime.datetime.now()
                # dd/mm/YY H:M:S
                dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
                log.write(str(listaResultados) + dt_string + '\n')
                listaResultados = []
            print("tiempo hasta siguiente log:" + str(tiempo))
            
            # caso en el que la respuesta no encuentra tweets o no hay mas tweets
            if (resultado[0] == 0) or ((resultado[0] > 0) and resultado[1] == ''):
                break
    
    print ("Tweets totales: " + str(total))
    print("Fin Hashtag: " + str(hashtag) + "\\\\\\\\\\\\ \n\n")

    return (total, hashtag)
    
    
    
            

In [3]:
#agregar que tipo de tweet es y a quien responden si es necesario
import tweepy
import csv
import pandas as pd
import json
from subprocess import check_output
import time
import math

#variables
entrada = ""
next_token = None
entradaizq = '' 
entradader = ''
hashtags = ''
total = 0
startDate = ''
endDate = ''
inicio = 0
fin = 0
resultado = ''
url = 'curl "https://api.twitter.com/2/tweets/search/all?'


f = open("./accountV2.txt", "r")
Bearer = f.readline().rstrip('\n')
f.close()


for l in hashtagsList:
    resultado = TweetList(l[0], l[1], l[2], 20000, Bearer)

log.close()
errorlog.close()

    


witter.com/mbtorregrosa/status/1386267731826298883/photo/1', 'display_url': 'pic.twitter.com/kArE8W4kiB'}, {'start': 79, 'end': 102, 'url': 'https://t.co/kArE8W4kiB', 'expanded_url': 'https://twitter.com/mbtorregrosa/status/1386267731826298883/photo/1', 'display_url': 'pic.twitter.com/kArE8W4kiB'}], 'mentions': [{'start': 3, 'end': 16, 'username': 'mbtorregrosa'}], 'hashtags': [{'start': 56, 'end': 78, 'tag': 'DemocraciaOFascismo4M'}]}

     author_id: 949716874392195074
     id: 1386267825556312068
       created_at: 25-04-2021T10:36:19.000Z
       referenced_tweets: [{'type': 'retweeted', 'id': '1386266422188331014'}]
       lang: es

       entities: {'urls': [{'start': 77, 'end': 100, 'url': 'https://t.co/3M0Di04vmp', 'expanded_url': 'https://twitter.com/CensoredJules/status/1386266422188331014/photo/1', 'display_url': 'pic.twitter.com/3M0Di04vmp'}, {'start': 77, 'end': 100, 'url': 'https://t.co/3M0Di04vmp', 'expanded_url': 'https://twitter.com/CensoredJules/status/1386266422188331